In [1]:
from read_images import create_image_dataset
MAIN_FOLDER_PATH="/home/faheem/Learning/Tensorflow/repo/tf_learning/fourth_week/EuroSAT_RGB"
img_dataset_object = create_image_dataset(MAIN_FOLDER_PATH)
train_data, test_data, train_labels,test_labels = img_dataset_object.load_dataset()

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense


2023-06-24 21:42:02.102746: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-24 21:42:02.947304: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
train_data.shape

(24300, 64, 64, 3)

In [7]:
def get_new_model(input_shape):
    model = Sequential([
        Conv2D(filters=16,kernel_size=(3,3),activation='relu',padding='SAME',name='conv_1',input_shape=(64,64,3)),
        Conv2D(filters=8,kernel_size=(3,3),activation='relu',padding='SAME',name='conv_2'),
        MaxPool2D(pool_size=(8,8),name='pool_1'),
        Flatten(),
        Dense(units=32,activation='relu',name='dense_1'),
        Dense(units=10,activation='softmax',name='dense_2')
    ])
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    return model

In [8]:
model = get_new_model((64,64,3))

2023-06-24 21:42:07.605657: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-24 21:42:07.648087: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-24 21:42:07.648371: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_1 (Conv2D)             (None, 64, 64, 16)        448       
                                                                 
 conv_2 (Conv2D)             (None, 64, 64, 8)         1160      
                                                                 
 pool_1 (MaxPooling2D)       (None, 8, 8, 8)           0         
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 32)                16416     
                                                                 
 dense_2 (Dense)             (None, 10)                330       
                                                                 
Total params: 18,354
Trainable params: 18,354
Non-traina

In [10]:
def get_test_accuracy(model,x_test,y_test):
    test_loss,test_acc = model.evaluate(x=x_test,y=y_test,verbose=0)
    print('accuracy: {acc:0.3f}'.format(acc=test_acc))

In [11]:
test_data.shape

(2700, 64, 64, 3)

In [12]:
get_test_accuracy(model,test_data,test_labels)

ValueError: in user code:

    File "/home/faheem/anaconda3/envs/tf_2/lib/python3.10/site-packages/keras/engine/training.py", line 1852, in test_function  *
        return step_function(self, iterator)
    File "/home/faheem/anaconda3/envs/tf_2/lib/python3.10/site-packages/keras/engine/training.py", line 1836, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/faheem/anaconda3/envs/tf_2/lib/python3.10/site-packages/keras/engine/training.py", line 1824, in run_step  **
        outputs = model.test_step(data)
    File "/home/faheem/anaconda3/envs/tf_2/lib/python3.10/site-packages/keras/engine/training.py", line 1791, in test_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/home/faheem/anaconda3/envs/tf_2/lib/python3.10/site-packages/keras/engine/training.py", line 1149, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/home/faheem/anaconda3/envs/tf_2/

: 

In [22]:
def get_checkpoint_every_epoch():
    return tf.keras.callbacks.ModelCheckpoint(filepath='checkpoints_every_epoch/checkpoint_{epoch:03d}',save_best_only=True,mode='max',save_freq='epoch')

def get_checkpoint_best_only():
    return tf.keras.callbacks.ModelCheckpoint(filepath='checkpoints_best_only/checkpoint',monitor='val_acc',save_best_only=True)

def get_early_stopping():
    return tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=3)


In [23]:
checkpoint_every_epoch = get_checkpoint_every_epoch()
checkpoint_best_only = get_checkpoint_best_only()
early_stopping = get_early_stopping()

In [1]:
callbacks = [checkpoint_every_epoch, checkpoint_best_only, early_stopping]
model.fit(train_data,train_labels,epochs=50,validation_data=(test_data,test_labels),callbacks=callbacks,batch_size=4)

NameError: name 'checkpoint_every_epoch' is not defined